In [22]:
import os
import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import InceptionResnetV1, MTCNN
from scipy.spatial.distance import cosine
from datetime import datetime
import time
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, db
from torchvision import transforms
from google.cloud import storage
from io import BytesIO
import json
import pickle
shown_messages = set()


In [23]:
cred = credentials.Certificate(r"C:\Users\enesb\Documents\GitHub\Real-Time-Face-Recognition-and-Attendance-System\real-time-attendance-sys-13a15-firebase-adminsdk-fbsvc-0b0ea93420.json")
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://real-time-attendance-sys-13a15-default-rtdb.europe-west1.firebasedatabase.app/'
    })

In [24]:
# Google Cloud Storage Bağlantısı
storage_client = storage.Client.from_service_account_json(
    r"C:\Users\enesb\Documents\GitHub\Real-Time-Face-Recognition-and-Attendance-System\real-time-attendance-460605-4367d4b382a9.json"
)
bucket = storage_client.bucket("dataset-aee")

In [25]:
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

In [26]:
if os.path.exists('known_faces.pkl'):
    with open('known_faces.pkl', 'rb') as f:
        known_faces = pickle.load(f)
    print("Cache'den known_faces yüklendi.")
else:
    known_faces = {}

CACHE_FILE = 'cached_faces.json'
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, 'r') as f:
        processed_files = set(json.load(f))
else:
    processed_files = set()

new_processed_files = set()
person_images = {}

blobs = bucket.list_blobs(prefix="dataset/")
for blob in blobs:
    if blob.name.endswith(".jpg") and blob.name not in processed_files:
        path_parts = blob.name.split("/")
        if len(path_parts) >= 5:
            fakulte, bolum, sinif, person_name = path_parts[1], path_parts[2], path_parts[3], path_parts[4]
            img_bytes = blob.download_as_bytes()
            img = Image.open(BytesIO(img_bytes)).convert("RGB")

            full_key = f"{fakulte}/{bolum}/{sinif}/{person_name}"

            if full_key not in person_images:
                person_images[full_key] = []
            person_images[full_key].append(img)
            new_processed_files.add(blob.name)

for person, images in person_images.items():
    embeddings = []
    for img in images:
        faces = mtcnn(img)
        if faces is not None:
            face_tensor = faces[0].unsqueeze(0)
            embedding = model(face_tensor).detach().numpy()
            embeddings.append(embedding)
    if embeddings:
        if len(embeddings) == 0:
            print(f"'{person}' için yüz bulunamadı.")
        known_faces[person] = np.mean(embeddings, axis=0)

Cache'den known_faces yüklendi.


In [27]:
all_processed = processed_files.union(new_processed_files)
with open(CACHE_FILE, 'w') as f:
    json.dump(list(all_processed), f)

with open('known_faces.pkl', 'wb') as f:
    pickle.dump(known_faces, f)

print("GCS üzerinden veritabanı oluşturuldu:", known_faces.keys())

GCS üzerinden veritabanı oluşturuldu: dict_keys(['Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Elifsu_Karayiğit', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Feyzanur_Sanrı', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Ghayad_Basmah_Ji', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Mustafa_Kerem_Duzdemir', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Rabia_Hasim', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Songül_Genc', 'Mühendislik ve Doğa Bilimleri Fakültesi/Bilgisayar Mühendisliği/4. Sınıf/Tuba_Calci', 'Mühendislik ve Doğa Bilimleri Fakültesi/Yazılım Mühendisliği/4. Sınıf/Abdullatif_Akkaya', 'Mühendislik ve Doğa Bilimleri Fakültesi/Yazılım Mühendisliği/4. Sınıf/Abdussamet_Oguz', 'Mühendislik ve Doğa Bilimleri Fakültesi/Yazılım Mühendisliği/4. Sınıf/Ahmet_Furkan_Aylac', 'Mühendislik ve Doğa Bilimleri Fakültesi

In [28]:
def compare_faces(embedding, known_faces):
    min_distance = float('inf')
    name = None
    for known_name, known_embedding in known_faces.items():
        distance = cosine(embedding.flatten(), known_embedding.flatten())
        if distance < min_distance:
            min_distance = distance
            name = known_name
    if min_distance < 0.3:
        return name
    else:
        return "Tanımlanamayan Kişi"

def sanitize(s):
    return s.replace('.', '_').replace('$', '_').replace('#', '_').replace('[', '_').replace(']', '_').replace('/', '_')

def add_attendance(full_key):    
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_key = now.strftime("%Y-%m-%d %H:%M:%S")
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

    path_parts = full_key.split("/")
    if len(path_parts) == 4:
        fakulte, bolum, sinif, isim = path_parts

        base_path = f"attendance/{sanitize(fakulte)}/{sanitize(bolum)}/{sanitize(sinif)}/{sanitize(isim)}"
        ref = db.reference(base_path)

        # Var olan kayıtları al
        existing_data = ref.get()
        count_today = 0

        if existing_data:
            for key in existing_data:
                if key.startswith(date_str):
                    count_today += 1

        if count_today >= 10:
            msg_key = f"{isim}_limit"
            if msg_key not in shown_messages:
                print(f"{isim} için günlük 10 kayıt sınırına ulaşıldı.")
                shown_messages.add(msg_key)
            return

        # Yeni yoklama verisini ekle
        ref.child(time_key).set({
            'status': 'Present',
            'timestamp': timestamp
        })

        msg_key = f"{isim}_success"
        if msg_key not in shown_messages:
            print(f"{isim} için yeni yoklama eklendi.")
            shown_messages.add(msg_key)



recorded_names = set()
verification_counts = defaultdict(int)
verification_threshold = 3

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kamera açılamadı!")
    exit()

try:
    prev_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kare okunamadı!")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_frame = Image.fromarray(frame_rgb)

        boxes, _ = mtcnn.detect(pil_frame)
        if boxes is not None:
            draw = ImageDraw.Draw(pil_frame)
            for box in boxes:
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=3)
                face = pil_frame.crop((box[0], box[1], box[2], box[3]))
                face = face.resize((160, 160))

                face_tensor = transforms.ToTensor()(face).unsqueeze(0)
                face_embedding = model(face_tensor).detach().numpy()

                full_key = compare_faces(face_embedding, known_faces)
                isim = full_key.split('/')[-1] if '/' in full_key else full_key

                font = ImageFont.truetype("arial.ttf", 30)
                draw.text((box[0], box[1] - 30), isim, fill=(255, 0, 0), font=font)

                if full_key != "Tanımlanamayan Kişi":
                    verification_counts[full_key] += 1
                    if verification_counts[full_key] >= verification_threshold:
                        add_attendance(full_key)
                        verification_counts[full_key] = 0
                else:
                    verification_counts = defaultdict(int)

        frame_with_boxes = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)

        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        cv2.putText(frame_with_boxes, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Face Recognition', frame_with_boxes)

        if cv2.waitKey(1) & 0xFF == ord('w'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

Enes_Berk_Demirci için günlük 10 kayıt sınırına ulaşıldı.
